In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
pd.options.mode.chained_assignment = None

In [2]:
from sklearn.metrics import confusion_matrix,average_precision_score,auc,roc_curve,roc_auc_score,precision_recall_curve

In [4]:
df=pd.read_csv('elnino.csv')
#data.copy()
df.shape

(9900, 15)

In [4]:
#df=pd.read_csv('bodyfat.csv')
df = dat.sample(n=9000)
data=df.copy()
data.shape

(9000, 13)

In [5]:
data.columns

Index(['Unnamed: 0', 'Observation', ' Year', ' Month', ' Day', ' Date',
       ' Latitude', ' Longitude', ' Zonal Winds', ' Meridional Winds',
       ' Humidity', ' Air Temp', ' Sea Surface Temp'],
      dtype='object')

In [6]:
N = df.shape[0]      # No of records in data
k = np.int(np.ceil(N*0.1))  # No of outliers to be inserted
k_rand_ind = random.sample(range(0,N),k)
k_rand_pt = df.iloc[k_rand_ind,:]
k_rand_pt1 = k_rand_pt.copy()
z_rand_list = []
y_features = [' Humidity']

for i in range(k):
    # Randomely choose a record from data
    z_rand = pd.DataFrame()
    z_rand = k_rand_pt.iloc[i:i+1,:]

    # Use the algorithm to exhange y_i with y_j
    p = np.int(np.ceil(min(50, N/4)))
    p_rand_ind = random.sample(range(0,N),p)
    p_rand_pt = df.iloc[p_rand_ind,:]
    for col in y_features:
        p_rand_pt['d'] = np.sqrt((p_rand_pt[col]-z_rand[col].values[0])**2)
        z_max_d = p_rand_pt[p_rand_pt.d==max(p_rand_pt.d)].head(1)
        z_rand.loc[:, col]=z_max_d[col].values

    z_rand_list.append(z_rand)
    
z_new = pd.concat(z_rand_list, ignore_index=True)  # Final outliers data
data['Train']=0
z_new['Train']=1
final_df = pd.concat([data, z_new], ignore_index=True)   # final data with outliers inserted
final_df.tail()

,Unnamed: 0,Observation,Year,Month,Day,Date,Latitude,Longitude,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp,Train
9895,17755,17756,93,6,27,930627,0.01,-155.01,-3.7,-0.4,95.3,28.33,28.68,1
9896,16701,16702,98,6,10,980610,0.00,146.99,-2.1,-0.1,70.0,28.90,29.51,1
9897,42692,42693,95,4,25,950425,2.04,-110.10,-5.2,3.5,69.5,25.95,26.92,1
9898,159176,159177,96,8,13,960813,8.06,-94.95,-0.3,2.1,68.6,26.27,27.54,1
9899,167314,167315,92,12,8,921208,-7.98,164.97,-3.1,-0.4,97.4,29.11,29.87,1


In [52]:
final_df.to_csv('cod_elnino.csv')

In [5]:
df.shape

(9900, 15)

In [6]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Observation', ' Year', ' Month', ' Day',
       ' Date', ' Latitude', ' Longitude', ' Zonal Winds', ' Meridional Winds',
       ' Humidity', ' Air Temp', ' Sea Surface Temp', 'Outlier'],
      dtype='object')

In [6]:
x_test=df[[' Year', ' Month', ' Day', ' Date',
       ' Latitude', ' Longitude', ' Zonal Winds', ' Meridional Winds', ' Air Temp', ' Sea Surface Temp']]
y_test=df[[' Humidity']]
test_o=df[[' Year', ' Month', ' Day', ' Date',
       ' Latitude', ' Longitude', ' Zonal Winds', ' Meridional Winds',
       ' Humidity', ' Air Temp', ' Sea Surface Temp']]

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scale=scaler.fit(x_test)
x_scaled=scaler.transform(x_test)

In [8]:
scaled_dat=scaler.fit(test_o)
scaled_data=scaler.transform(test_o)

In [11]:
x_test.shape

(9900, 10)

In [12]:
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras.optimizers import RMSprop 
from keras.callbacks import EarlyStopping 

#define base model
model = Sequential()
model.add(Dense(128, kernel_initializer='normal',input_dim = 10, activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1408      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 100,481
Trainable params: 100,481
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(
   x_scaled, 
   y_test,    
   batch_size=128, 
   epochs = 500, 
   verbose = 1, 
   validation_split = 0.2, 
   callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)]
)

Epoch 1/500
62/62 [==============================] - 2s 22ms/step - loss: 67.6694 - mean_absolute_error: 67.6694 - val_loss: 13.1674 - val_mean_absolute_error: 13.1674
Epoch 2/500
62/62 [==============================] - 0s 7ms/step - loss: 10.0444 - mean_absolute_error: 10.0444 - val_loss: 10.7162 - val_mean_absolute_error: 10.7162
Epoch 3/500
62/62 [==============================] - 0s 7ms/step - loss: 7.1905 - mean_absolute_error: 7.1905 - val_loss: 9.5440 - val_mean_absolute_error: 9.5440
Epoch 4/500
62/62 [==============================] - 0s 7ms/step - loss: 5.7846 - mean_absolute_error: 5.7846 - val_loss: 9.0782 - val_mean_absolute_error: 9.0782
Epoch 5/500
62/62 [==============================] - 0s 7ms/step - loss: 4.9221 - mean_absolute_error: 4.9221 - val_loss: 8.7816 - val_mean_absolute_error: 8.7816
Epoch 6/500
62/62 [==============================] - 0s 7ms/step - loss: 4.3581 - mean_absolute_error: 4.3581 - val_loss: 8.6109 - val_mean_absolute_error: 8.6109
Epoch 7/500
6

Epoch 51/500
62/62 [==============================] - 0s 8ms/step - loss: 3.0265 - mean_absolute_error: 3.0265 - val_loss: 8.1939 - val_mean_absolute_error: 8.1939
Epoch 52/500
62/62 [==============================] - 0s 7ms/step - loss: 2.9936 - mean_absolute_error: 2.9936 - val_loss: 8.2082 - val_mean_absolute_error: 8.2082
Epoch 53/500
62/62 [==============================] - 0s 7ms/step - loss: 2.9251 - mean_absolute_error: 2.9251 - val_loss: 8.2843 - val_mean_absolute_error: 8.2843
Epoch 54/500
62/62 [==============================] - 0s 7ms/step - loss: 2.9551 - mean_absolute_error: 2.9551 - val_loss: 8.2066 - val_mean_absolute_error: 8.2066
Epoch 55/500
62/62 [==============================] - 0s 7ms/step - loss: 2.9553 - mean_absolute_error: 2.9553 - val_loss: 8.1955 - val_mean_absolute_error: 8.1955
Epoch 56/500
62/62 [==============================] - 0s 7ms/step - loss: 2.9031 - mean_absolute_error: 2.9031 - val_loss: 8.3251 - val_mean_absolute_error: 8.3251
Epoch 57/500
62/

In [14]:
from sklearn.metrics import r2_score, mean_squared_error
mse=mean_squared_error(y_test, model.predict(x_scaled))
r2 =r2_score(y_test, model.predict(x_scaled))
print('R2_score:',r2)
print('Mean_squared_error:',mse)

R2_score: 0.18702710797224842
Mean_squared_error: 32.32278375337778


In [16]:
test_outcome=df.copy()
y_pred_test=np.round_(model.predict(x_scaled),decimals=4)
test_outcome['test_prediction']=y_pred_test
diff=np.absolute(test_outcome['test_prediction']-test_outcome[' Humidity'])
#norm=np.sqrt(difference)
#weight=np.max(r2,0)
#weighted_score=np.round(norm*weight,decimals=2)
test_outcome['out_score']=diff
test_result=test_outcome[[' Humidity','test_prediction','out_score']]
test_result.tail()

,Humidity,test_prediction,out_score
9895,95.3,78.180199,17.119801
9896,70.0,78.926300,8.926300
9897,69.5,82.280296,12.780296
9898,68.6,82.344002,13.744002
9899,97.4,80.614601,16.785399


In [46]:
from pyod.models.iforest import IForest
clf=IForest()
clf.fit(test_result[['out_score']])

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [47]:
y_pred_cod=clf.labels_
y_prob_cod=clf.predict_proba(test_result[['out_score']])[:,1]
y_true=df[['Outlier']]
confusion_matrix(y_true,y_pred_cod)

array([[8831,  169],
       [  80,  820]], dtype=int64)

In [48]:
#Isolation Forest results
fpr,tpr,threshold=roc_curve(y_true,y_prob_cod,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_cod))
print('auc: %.4f' % auc(fpr,tpr))

Average prcision: 0.7635
auc: 0.9930


In [29]:
##KNN based COD detection
from pyod.models.knn import KNN
kn=KNN(metric='minkowski',n_neighbors=20)
kn.fit(scaled_data)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=20, p=2,
  radius=1.0)

In [30]:
##KNN based COD detection
y_pred_kn=kn.labels_
y_prob_kn=kn.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_kn)

array([[8427,  573],
       [ 483,  417]], dtype=int64)

In [31]:
##KNN based COD detection result
fpr_kn,tpr_kn,threshold_kn=roc_curve(y_true,y_prob_kn,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_kn))
print('auc: %.4f' % auc(fpr_kn,tpr_kn))

Average prcision: 0.2439
auc: 0.8820


In [32]:
#LOF analysis
from pyod.models.lof import LOF
clf_lof=LOF(n_neighbors=20)
clf_lof.fit(scaled_data)

LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=False, p=2)

In [33]:
##KNN based COD detection
y_pred_lf=clf_lof.labels_
y_prob_lf=clf_lof.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_lf)

array([[8416,  584],
       [ 494,  406]], dtype=int64)

In [34]:
#LOF results
fpr_lf, tpr_lf, thresholds_lf = roc_curve(y_true, y_prob_lf,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_lf))
print('AUC_score: %.4f' % auc(fpr_lf, tpr_lf))

Average prcision: 0.2349
AUC_score: 0.8649


In [35]:
#COF analysis
from pyod.models.cof import COF
clf_cof=COF()
clf_cof.fit(scaled_data)

COF(contamination=0.1, method='fast', n_neighbors=None)

In [36]:
##COF based COD detection
y_pred_cf=clf_cof.labels_
y_prob_cf=clf_cof.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_cf)

array([[8364,  636],
       [ 546,  354]], dtype=int64)

In [37]:
#COF results
fpr_cf, tpr_cf, thresholds_cf = roc_curve(y_true, y_prob_cf,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_cf))
print('AUC_score: %.4f' % auc(fpr_cf, tpr_cf))

Average prcision: 0.1958
AUC_score: 0.8046


In [38]:
#IForest analysis
from pyod.models.iforest import IForest
clf_if=IForest()
clf_if.fit(scaled_data)

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [39]:
##IForest based COD detection
y_pred_if=clf_if.labels_
y_prob_if=clf_if.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_if)

array([[8210,  790],
       [ 700,  200]], dtype=int64)

In [40]:
#IForest results
fpr_if, tpr_if, thresholds_if = roc_curve(y_true, y_prob_if,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_if))
print('AUC_score: %.4f' % auc(fpr_if, tpr_if))

Average prcision: 0.1156
AUC_score: 0.6680


In [9]:
#COPOD analysis
from pyod.models.sos import SOS
clf_va=SOS()
clf_va.fit(test_o)

SOS(contamination=0.1, eps=1e-05, metric='euclidean', perplexity=4.5)

In [11]:
##COPOD based COD detection
y_pred_va=clf_va.labels_
y_prob_va=clf_va.predict_proba(test_o)[:,1]
y_true=df[['Outlier']]
confusion_matrix(y_true,y_pred_va)

array([[8427,  573],
       [ 483,  417]], dtype=int64)

In [12]:
#COPOD results
fpr_va, tpr_va, thresholds_va = roc_curve(y_true, y_prob_va,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_va))
print('AUC_score: %.4f' % auc(fpr_va, tpr_va))

Average prcision: 0.2439
AUC_score: 0.8180


In [49]:
#ROC analysis
import plotly.graph_objects as go

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1)

fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f'NCOD:1(AUC={auc(fpr, tpr):.3f})', mode='lines',marker_color='crimson'))
fig.add_trace(go.Scatter(x=fpr_if, y=tpr_if, name=f'COPOD(AUC={auc(fpr_va, tpr_va):.3f})', mode='lines',marker_color='hotpink'))
fig.add_trace(go.Scatter(x=fpr_kn, y=tpr_kn, name=f'KNN(AUC={auc(fpr_kn, tpr_kn):.3f})', mode='lines',marker_color='yellow'))
fig.add_trace(go.Scatter(x=fpr_lf, y=tpr_lf, name=f'LOF(AUC={auc(fpr_lf, tpr_lf):.3f})', mode='lines'))
fig.add_trace(go.Scatter(x=fpr_cf, y=tpr_cf, name=f'COF(AUC={auc(fpr_cf, tpr_cf):.3f})', mode='lines'))
fig.add_trace(go.Scatter(x=fpr_if, y=tpr_if, name=f'IForest(AUC={auc(fpr_if, tpr_if):.3f})', mode='lines',marker_color='hotpink'))



fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=600)
fig.show()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(data)

In [ ]:
ts=pd.DataFrame(projections)
ts.columns=['tsne_feature1','tsne_feature2']

In [ ]:
y_out=df['Out'].astype('category')

In [ ]:
ts['outlier']=y_out

In [ ]:
import plotly.express as px
fig = px.scatter(ts, x='tsne_feature1', y='tsne_feature2',color='outlier')
fig.show()